<a href="https://colab.research.google.com/github/mikel-k-khui/GDG-PyYYC-Python-Bootcamp/blob/main/Google_Drive_Snapshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Snapshot Generator

### Setup
This script will connect Colabatory to Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: ignored

### Operation Templates
This script will take a snapshot of the current operation templates in the PostgreSQL database and save them to Google Drive.

For each operation template, it will calculate the average duration of operations created from this template, and add it to the file.

In [ ]:
import pandas as pd
from sqlalchemy import MetaData, create_engine, extract, select, text
from sqlalchemy.sql import func

engine = create_engine("postgresql://install-app-viewer:ZhmAK8>&TYu+A*d9@35.203.77.149:5432/install-app-link-testing")
connection = engine.connect()
metadata = MetaData()
metadata.reflect(bind=connection)
template_operation = metadata.tables.get("template_operation")
templates = connection.execute(
    select(template_operation)
      .order_by(template_operation.c.created_on.asc())
    ).all()

df = pd.DataFrame(templates).set_index('id')

averages = []
operation = metadata.tables.get("operation")
for template in templates:
  # Find the total time by taking the difference between epochs (epoch = total # of seconds since 1970)
  avg_duration = connection.execute(
      select(
                func.avg((extract('epoch', operation.c.current_end) - extract('epoch', operation.c.current_start)) / 60)
          )
        .where(operation.c.template_fk == template.id)
        .where(operation.c.current_start.is_not(None), operation.c.current_end.is_not(None))
  ).scalar_one() or 0
  averages.append(avg_duration)

df['avg_duration_minutes'] = averages
display(df)

,created_on,updated_on,title,description,avg_duration_minutes
id,,,,,
1,2023-04-19 17:51:36.743776+00:00,2023-04-19 17:51:36.743776+00:00,Chang-Fisher,Magazine truth stop whose group through despit...,120.0000000000000000
2,2023-04-19 17:51:38.814393+00:00,2023-04-19 17:51:38.814393+00:00,"Mcgrath, Patrick and Barrera",Role together range line. Government first pol...,300.0000000000000000
3,2023-04-19 17:51:40.919588+00:00,2023-04-19 17:51:40.919588+00:00,"Woods, Grimes and Green",Behavior style report. Ability management test...,0


In [ ]:
# save as .csv file
import os
import datetime

SNAPSHOT_DIR = 'snapshots'
file_name = f'operation_template_{datetime.datetime.now().strftime("%Y-%m-%d_%H%M")}.csv'

# TODO: currently this saves to your local hard drive, but it should save to Google drive
file_dir = os.path.join(os.getcwd(), SNAPSHOT_DIR) # f"/content/gdrive/{SNAPSHOT_DIR}"
if not os.path.exists(file_dir):
  os.makedirs(file_dir)

file_path = os.path.join(file_dir, file_name)
df.to_csv(file_path, index=False)
print(f"Saved to {file_path}.")

Saved to C:\Users\okos0\Projects\install-app-data\snapshots\operation_template_2023-04-27_1136.csv.
